In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms
import os
import sys
from pathlib import Path
import timm
import json

/home/s180366/raid/flooded-areas-ai-pytorch/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

current_cuda_device = None
print(f"is cuda available: {torch.cuda.is_available()}")
print(f"cuda device count: {torch.cuda.device_count()}")
current_cuda_device = torch.cuda.current_device()
print(f"current cuda device: {current_cuda_device}")
print(f"current cuda device name: {torch.cuda.get_device_name(current_cuda_device)}")

device = (
    "cuda" if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available()
    else "cpu"
)

print(f"Using {device} device")

is cuda available: True
cuda device count: 1
current cuda device: 0
current cuda device name: NVIDIA A40
Using cuda device


In [3]:
modules_path = os.path.abspath("..")
if modules_path not in sys.path:
    sys.path.append(modules_path)

from data.FloodNet import FloodNetDataset

In [4]:
transform = transforms.Compose([
    transforms.Resize(299),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # Normalize with ImageNet mean and std
                         std=[0.229, 0.224, 0.225])
])

In [5]:
with open("../data/classes.json", "r") as j:
    classes = json.load(j)
# Create train dataset
train_images_dir = Path("../data/FloodNet_dataset/train/image")
train_csv_file = Path("../data/flood_train_rel_paths.csv")
train_data = FloodNetDataset(annotations_file=train_csv_file, img_dir=train_images_dir, classes=classes, transform=transform)

# Create test dataset
test_images_dir = Path("../data/FloodNet_dataset/test/image")
test_csv_file = Path("../data/flood_test_rel_paths.csv")
test_data = FloodNetDataset(annotations_file=test_csv_file, img_dir=test_images_dir, classes=classes, transform=transform)

In [6]:
print(train_data.get_class_dict())
print(train_data.get_class_names())

{'Background': 0, 'Building-flooded': 1, 'Building-non-flooded': 2, 'Road-flooded': 3, 'Road-non-flooded': 4, 'Water': 5, 'Tree': 6, 'Vehicle': 7, 'Pool': 8, 'Grass': 9}
['Background', 'Building-flooded', 'Building-non-flooded', 'Road-flooded', 'Road-non-flooded', 'Water', 'Tree', 'Vehicle', 'Pool', 'Grass']


In [7]:
batch_size = 64

train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [ ]:
for X, y in train_dataloader:
    # print(X)
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

In [8]:
model = timm.create_model('xception', pretrained=True)
model.eval()

/home/s180366/raid/flooded-areas-ai-pytorch/.venv/lib/python3.8/site-packages/timm/models/_factory.py:126: UserWarning: Mapping deprecated model name xception to current legacy_xception.
  model = create_fn(
Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-cadene/xception-43020ad28.pth" to /home/s180366/.cache/torch/hub/checkpoints/xception-43020ad28.pth


Xception(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act2): ReLU(inplace=True)
  (block1): Block(
    (skip): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
    (skipbn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (rep): Sequential(
      (0): SeparableConv2d(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
        (pointwise): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): SeparableConv2d(
        (conv1): Conv

In [9]:
# modyfing Xception for binary classification

# Original final layer: model.fc = nn.Linear(2048, 1000)
model.fc = nn.Linear(model.num_features, 1)  # Binary output

In [10]:
model = model.to(device)
print(model)

Xception(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act2): ReLU(inplace=True)
  (block1): Block(
    (skip): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
    (skipbn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (rep): Sequential(
      (0): SeparableConv2d(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
        (pointwise): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): SeparableConv2d(
        (conv1): Conv

In [13]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [14]:
def train(dataloader: DataLoader, model, loss_fn, optimizer):
    print("Training...")
    size = len(dataloader.dataset)
    current = 0
    model.train()
    # print(f"num batches: {len(dataloader)}")
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device).float().unsqueeze(1)
        # compute prediction error
        optimizer.zero_grad()

        outputs = model(X)

        loss = loss_fn(outputs, y)

        # backpropagation
        loss.backward()
        optimizer.step()

        # if batch % 100 == 0:
        loss, current = loss.item(), current + len(X)
        print(f"batch: {batch}, loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [15]:
def test(dataloader: DataLoader, model):
    print("Testing...")
    size = len(dataloader.dataset)
    # print(f"size: {size}")
    num_batches = len(dataloader)
    # print(f"num batches: {num_batches}")
    model.eval()
    correct = 0
    total = 0
    with torch.inference_mode():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)

            outputs = model(X)
            preds = torch.sigmoid(outputs) > 0.5
            correct += (preds.squeeze().long() == y).sum().item()
            total += y.size(0)
    # print(f"Test error: \n Accuracy: {(100*correct):>0.1f}%, avg loss: {test_loss:>8f} \n")
    print(f"Test error: \n Accuracy: {correct / total * 100:.2f}%")

In [16]:
epochs = 5

for epoch in range(epochs):
    print(f"-------------- Epoch {epoch+1} --------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model)
print("Done!")

-------------- Epoch 1 --------------
Training...
batch: 0, loss: 0.689186 [   64/ 1445]
batch: 1, loss: 0.509934 [  128/ 1445]
batch: 2, loss: 0.362960 [  192/ 1445]
batch: 3, loss: 0.245583 [  256/ 1445]
batch: 4, loss: 0.305148 [  320/ 1445]
batch: 5, loss: 0.253838 [  384/ 1445]
batch: 6, loss: 0.382723 [  448/ 1445]
batch: 7, loss: 0.255758 [  512/ 1445]
batch: 8, loss: 0.333393 [  576/ 1445]
batch: 9, loss: 0.174152 [  640/ 1445]
batch: 10, loss: 0.205496 [  704/ 1445]
batch: 11, loss: 0.236530 [  768/ 1445]
batch: 12, loss: 0.197563 [  832/ 1445]
batch: 13, loss: 0.234518 [  896/ 1445]
batch: 14, loss: 0.160373 [  960/ 1445]
batch: 15, loss: 0.296845 [ 1024/ 1445]
batch: 16, loss: 0.100088 [ 1088/ 1445]
batch: 17, loss: 0.156400 [ 1152/ 1445]
batch: 18, loss: 0.163055 [ 1216/ 1445]
batch: 19, loss: 0.121727 [ 1280/ 1445]
batch: 20, loss: 0.061935 [ 1344/ 1445]
batch: 21, loss: 0.140658 [ 1408/ 1445]
batch: 22, loss: 0.308514 [ 1445/ 1445]
Testing...
Test error: 
 Accuracy: 93.53

KeyboardInterrupt: 

In [ ]:
from torchinfo import summary
summary(model, input_size=[1, 3, 299, 299])